In [ ]:
import numpy as np                                     # 넘파이 패키지
import pandas as pd                                    # 판다스 패키지
import matplotlib.pyplot as plt                        # 그래프 패키지
from sklearn.model_selection import train_test_split   # 데이터 셋 분할

from sklearn.linear_model import LinearRegression      # 선형 회귀 패키지
from sklearn.linear_model import Ridge                 # 릿지 선형 회귀 패키지
from sklearn.preprocessing import PolynomialFeatures   # 특성 변환기 패키지

from keras.models import Sequential, Model
from keras import optimizers
import tensorflow as tf

from keras.layers import Dense, Input, Flatten, Concatenate, LSTM, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from keras.layers import concatenate

from sklearn.metrics import mean_absolute_error        # 평균 절대값 에러(모델 평가 지수)
from sklearn.metrics import mean_squared_error         # 평균 제곱근 에러(모델 평가 지수)

In [ ]:
# 코랩과 구굴 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')   # content는 내 드라이브 디렉토리입니다.

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RNN/data/mokpo_data.csv')
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30624 entries, 0 to 30623
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기온      30624 non-null  float64
 1   풍속      30624 non-null  float64
 2   습도      30624 non-null  int64  
 3   기압      30624 non-null  float64
 4   일조      30624 non-null  float64
 5   일사      30624 non-null  float64
 6   전운량     30624 non-null  int64  
 7   발전량     30624 non-null  float64
dtypes: float64(6), int64(2)
memory usage: 1.9 MB
None
    기온   풍속  습도      기압   일조  일사   전운량  발전량
0 -3.6  1.9  66  1027.7  0.0  0.0    9  0.0
1 -4.4  2.3  66  1027.6  0.0  0.0    6  0.0
2 -4.1  2.1  65  1028.1  0.0  0.0    9  0.0
3 -4.0  1.1  63  1028.0  0.0  0.0    9  0.0
4 -4.2  1.3  63  1027.5  0.0  0.0    9  0.0


In [ ]:
data = np.array(data)          # 데이터 프레임을 배열로 변환
# 정규화 : 각 열 단위로 0 부터 1롤 변환
data = (data - np.min(data, 0)) / (np.max(data, 0) - np.min(data, 0))
print(data.shape)              # 데이터 스케일 보기
print(len(data) * 0.8)         # 데이터 나누기 위해서

(30624, 8)
24499.2


In [ ]:
# 데이터셋 만들기(주의 : 다음 시간 예측입니다)
x_data = data[ : -1, : ]             # 입력 데이터(마지막 샘플 전까지)
y_data = data[1 : , -1]              # 타겟 데이터(다음날 부터 마지막 샘플까지)
print(x_data.shape)
print(y_data.shape)
print(y_data[:5])

(30623, 8)
(30623,)
[0. 0. 0. 0. 0.]


In [ ]:
# 데이터 셋 만들기
x_train = x_data[ : 24499 ,  : ]         # 학습 데이터
y_train = y_data[ : 24499]

x_test = x_data[24499 : ,  : ]          # 테스트 데이터
y_test = y_data[24499 :]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(24499, 8)
(24499,)
(6124, 8)
(6124,)


degree=4

In [ ]:
poly = PolynomialFeatures(degree=4, include_bias=False)   # 특성 변환기 모델
poly.fit(x_train)                                         # 3개의 특성을 이용하여 특성을 늘리는 학습

train_poly = poly.transform(x_train)                      # 여기서 훈련 셋 특성 변함
test_poly = poly.transform(x_test)                        # 테스트 셋도 훈련셋과 동일하게 특성 해야 함

print(train_poly.shape)                                   # 특성 변환 스케알

(24499, 494)


In [ ]:
lr = LinearRegression()                              # 선형 회귀 모델
lr.fit(train_poly, y_train)                    # 모델 훈련
print(lr.score(train_poly, y_train))           # 훈련 셋 평가
print(lr.score(test_poly, y_test))             # 테스트 셋 평가

0.9154697941620656
0.9012717124327756


In [ ]:
# 테스트 세트에 대한 예측 값
test_prediction = lr.predict(test_poly)
# 테스트 세트에 대한 평균 절댓값 오차를 계산합니다
mae = mean_absolute_error(y_test, test_prediction)
mse = mean_squared_error(y_test, test_prediction)
rmse = mse**0.5
print(mae)
print(mse)
print(rmse)

0.044103830193033586
0.006279905011638175
0.07924585169987244


##DLSTM
DNN + LSTM 병렬 모델


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RNN/data/mokpo_data.csv')
print(data.info())
print(data.head())
data = np.array(data)          # 데이터 프레임을 배열로 변환
# 정규화 : 각 열 단위로 0 부터 1롤 변환
data = (data - np.min(data, 0)) / (np.max(data, 0) - np.min(data, 0))
print(data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30624 entries, 0 to 30623
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기온      30624 non-null  float64
 1   풍속      30624 non-null  float64
 2   습도      30624 non-null  int64  
 3   기압      30624 non-null  float64
 4   일조      30624 non-null  float64
 5   일사      30624 non-null  float64
 6   전운량     30624 non-null  int64  
 7   발전량     30624 non-null  float64
dtypes: float64(6), int64(2)
memory usage: 1.9 MB
None
    기온   풍속  습도      기압   일조  일사   전운량  발전량
0 -3.6  1.9  66  1027.7  0.0  0.0    9  0.0
1 -4.4  2.3  66  1027.6  0.0  0.0    6  0.0
2 -4.1  2.1  65  1028.1  0.0  0.0    9  0.0
3 -4.0  1.1  63  1028.0  0.0  0.0    9  0.0
4 -4.2  1.3  63  1027.5  0.0  0.0    9  0.0
(30624, 8)


In [ ]:
poly = PolynomialFeatures(degree=4, include_bias=False)   # 특성 변환기 모델
poly.fit(data)                                            # 4개의 특성을 이용하여 특성을 늘리는 학습

data = poly.transform(data)                               # 모든 데이터 셋 변함

print(data.shape)                                         # 특성 변환 스케알

(30624, 494)


In [ ]:
# DNN 과 LSTM 데이터 셋을 맞추기 위해 ...
# DNN 데이터 셋
dataD = data[ 24 : , : ]          # 24 행 부터(25일 예측)
print(dataD.shape)

(30600, 494)


In [ ]:
xD_data = dataD[ : -1, : ]             # 입력 데이터
yD_data = dataD[1 : , -1]              # 타겟 데이터

print(xD_data.shape)                   # 스케알
print(yD_data.shape)
print(len(xD_data) * 0.8)

(30599, 494)
(30599,)
24479.2


In [ ]:
xD_train = xD_data[ : 24478 ,  : ]            # 학습 데이터
yD_train = yD_data[ : 24478]
xD_test = xD_data[24478 : ,  : ]              # 테스트 데이터
yD_test = yD_data[24478 :]

print(xD_train.shape)
print(yD_train.shape)
print(xD_test.shape)
print(yD_test.shape)

(24478, 494)
(24478,)
(6121, 494)
(6121,)


In [ ]:
# ㅣㄴ쓰 데이터 셋 샹성
xL_data = []                         # 입력데이터 리스트
yL_data = []                         # 출력데이터 리스트

for i in range(1, len(data) - 24) :  # 타임 스탬프 24 개
  x = data[i : i + 24, : ]
  y = data[i + 24, [-1]]
  xL_data.append(x)
  yL_data.append(y)

xL_data = np.array(xL_data)
yL_data = np.array(yL_data)

print(xL_data.shape)
print(yL_data.shape)
print(len(xL_data) * 0.8)

(30599, 24, 494)
(30599, 1)
24479.2


In [ ]:
xL_train = xL_data[ : 24478 ,  : ]         # 학습 데이터
yL_train = yL_data[ : 24478]

xL_test = xL_data[24478 : ,  : ]          # 테스트 데이터
yL_test = yL_data[24478 :]

print(xL_train.shape)
print(yL_train.shape)
print(xL_test.shape)
print(yL_test.shape)

(24478, 24, 494)
(24478, 1)
(6121, 24, 494)
(6121, 1)


In [ ]:
inputA = Input(shape=(494,))                             # 입력층
hidden1 = Dense(256, activation='relu')(inputA)          # 1 층
hidden2 = Dense(256, activation='relu')(hidden1)        # 2 층
hidden3 = Dense(256, activation='relu')(hidden2)        # 3 층
hidden4 = Dense(256, activation='relu')(hidden3)        # 4층
hidden4 = hidden4 + hidden2
hidden5 = Dense(256, activation='relu')(hidden4)        # 5층
hidden6 = Dense(256, activation='relu')(hidden5)        # 6층
hidden6 = hidden6 + hidden4
outputA = Dense(8, activation='linear')(hidden6)      # 출력 층
# ---------------------------------------------------------
inputB = Input(shape=(24, 494))
lstm_layer = LSTM(128)(inputB)
hidden1 = Dense(128, activation='relu')(lstm_layer)
hidden2 = Dense(32, activation='relu')(hidden1)
outputB = Dense(8, activation='linear')(hidden2)
# ---------------------------------------------------------
con = concatenate([outputA, outputB])
h1 = Dense(16, activation = "relu")(con)
result = Dense(1, activation = "linear")(h1)

DL_model = Model(inputs = [inputA, inputB], outputs = [result])
DL_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 494)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 256)                  126720    ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 256)                  65792     ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 256)                  65792     ['dense_1[0][0]']             
                                                                                              

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)
DL_model.compile(loss = 'mse', optimizer = optimizers.Adam(0.001), metrics=['accuracy'])

In [ ]:
hist = DL_model.fit(
                  [xD_train, xL_train], yD_train,
                  epochs = 10, batch_size = 10,
                  validation_data=([xD_test, xL_test], yD_test),
                  callbacks = [early_stopping]
                 )

Epoch 1/10
2448/2448 [==============================] - 37s 10ms/step - loss: 0.0022 - accuracy: 0.4847 - val_loss: 0.0056 - val_accuracy: 0.5533
Epoch 2/10
2448/2448 [==============================] - 21s 9ms/step - loss: 0.0014 - accuracy: 0.4847 - val_loss: 0.0012 - val_accuracy: 0.5533
Epoch 3/10
2448/2448 [==============================] - 23s 9ms/step - loss: 0.0012 - accuracy: 0.4847 - val_loss: 0.0010 - val_accuracy: 0.5533
Epoch 4/10
2448/2448 [==============================] - 24s 10ms/step - loss: 0.0012 - accuracy: 0.4847 - val_loss: 0.0010 - val_accuracy: 0.5533
Epoch 5/10
2448/2448 [==============================] - 22s 9ms/step - loss: 0.0011 - accuracy: 0.4847 - val_loss: 9.8089e-04 - val_accuracy: 0.5533
Epoch 6/10
2448/2448 [==============================] - 22s 9ms/step - loss: 0.0011 - accuracy: 0.4847 - val_loss: 8.7273e-04 - val_accuracy: 0.5533
Epoch 7/10
2448/2448 [==============================] - 22s 9ms/step - loss: 0.0010 - accuracy: 0.4847 - val_loss: 0.001

In [ ]:
predict = DL_model.predict([xD_test, xL_test])

mae = mean_absolute_error(yD_test, predict)
mse = mean_squared_error(yD_test, predict)
rmse = mse**0.5
print(mae)
print(mse)
print(rmse)

192/192 [==============================] - 1s 3ms/step
0.010489480847496787
0.000907176406368458
0.03011936928902161


In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'], 'r', label = 'loss')
plt.plot(hist.history['val_loss'], 'b', label = 'val_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 3))
plt.plot(yD_test[0 : 240], 'r', label = 'real data')   # 실제값 출력
plt.plot(predict[00 : 240], 'b', label = 'predict')    # 예측값 출력
plt.xlabel("time")
plt.ylabel("solar insolation amount")
plt.legend(loc='upper left')
plt.show()